In [ ]:
import numpy as np
import random

# Distance matrix
distance_matrix = np.array([
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
])

# Parameters
num_ants = 10
num_iterations = 50
evaporation_rate = 0.5
alpha = 1    # pheromone importance
beta = 2     # visibility importance

num_cities = distance_matrix.shape[0]
pheromone = np.ones((num_cities, num_cities))
visibility = 1 / (distance_matrix + np.eye(num_cities) * 1e10)

def calculate_route_length(route):
    return sum(distance_matrix[route[i]][route[(i + 1) % num_cities]] for i in range(num_cities))

best_route = None
best_distance = float('inf')

for _ in range(num_iterations):
    all_routes = []
    for _ in range(num_ants):
        route = []
        unvisited = list(range(num_cities))
        current_city = random.choice(unvisited)
        route.append(current_city)
        unvisited.remove(current_city)

        while unvisited:
            probabilities = []
            for next_city in unvisited:
                tau = pheromone[current_city][next_city] ** alpha
                eta = visibility[current_city][next_city] ** beta
                probabilities.append(tau * eta)
            probabilities = np.array(probabilities)
            probabilities /= probabilities.sum()

            next_city = random.choices(unvisited, weights=probabilities)[0]
            route.append(next_city)
            unvisited.remove(next_city)
            current_city = next_city

        all_routes.append(route)

    # Update pheromones
    pheromone *= (1 - evaporation_rate)
    for route in all_routes:
        length = calculate_route_length(route)
        for i in range(num_cities):
            a, b = route[i], route[(i + 1) % num_cities]
            pheromone[a][b] += 1 / length
            pheromone[b][a] += 1 / length

    # Check for best route
    for route in all_routes:
        length = calculate_route_length(route)
        if length < best_distance:
            best_distance = length
            best_route = route

print("Best route:", best_route)
print("Shortest distance:", best_distance)
